# Chatbot
- [reference] https://python.langchain.com/v0.2/docs/tutorials/chatbot/


## Quickstart

In [113]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)


In [114]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [115]:
load_dotenv()

True

In [116]:
model = ChatOpenAI(model='gpt-4o')
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x109efba90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10d30a3b0>, root_client=<openai.OpenAI object at 0x10bd949d0>, root_async_client=<openai.AsyncOpenAI object at 0x109efb250>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [117]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
store = {}
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [118]:
model.invoke([HumanMessage(content="Hi! I'm Bob")])
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in our conversation. Can I help you with anything else today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 11, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9bddfca6e2', 'id': 'chatcmpl-BcRDOl1GkinwoByMAFPFYC5yusDAt', 'finish_reason': 'stop', 'logprobs': None}, id='run-f4b16e0e-a4b5-4a53-bec0-7fd82bd1ffbd-0', usage_metadata={'input_tokens': 11, 'output_tokens': 33, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [119]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [120]:
[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [121]:
from operator import itemgetter
dc = {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "한국어",
    }
result = itemgetter("messages")
result(dc)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [122]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't have access to your personal information, including your name. If you tell me your name, I'll be happy to use it in our conversation!"

In [123]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="My name is hahahohohihig, Fuck you")],
        "language": "한국어",
        
    }
)
response.usage_metadata

{'input_tokens': 72,
 'output_tokens': 22,
 'total_tokens': 94,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [124]:
response.usage_metadata['total_tokens']

94

In [125]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Could you say Korean?")],
        "language": "한국어",
    }
)
response.content

'Yes, I can understand and generate text in Korean. How can I assist you with Korean?'

In [126]:
store.keys()

dict_keys([])

In [127]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages',
)

config = {'configurable': {'session_id': 'abc11'}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="what math problem did i ask?")], "language": "Korean"},
    config=config,
)

response.content

"I'm sorry, but I can't recall past interactions or know what specific math problem you asked previously. However, please feel free to describe or ask your math problem again, and I'll do my best to assist you with it!"

In [128]:
get_session_history('abc20')

InMemoryChatMessageHistory(messages=[])

In [129]:
for messages in get_session_history('abc20').messages:
    print(f'[{message.type.upper()}]: {message.content}')

In [130]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="지금까지 대화 기록 내용 보여줘")], "language": "Korean"},
    config=config,
)

response.content

'죄송하지만, 보안과 개인정보 보호를 위해 현재까지의 대화 기록을 보여드릴 수 없습니다. 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.'

In [132]:
trimmer.invoke(get_session_history('abc20').messages)

[]